# To err is human, but where is the error?
### Erronous Hamsters group project
### Analysis of the HCP Dataset

In [1]:
#load important libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import warnings
from nistats.hemodynamic_models import glover_hrf

C:\Users\Frauke\anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: 

 | Starting with Nilearn 0.7.0, all Nistats functionality has been incorporated into Nilearn's stats & reporting modules.
 | Nistats package will no longer be updated or maintained.

  


In [2]:
#@title Figure settings
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
#np.set_printoptions(threshold=sys.maxsize)
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")

## Basic Parameters

In [3]:
HCP_DIR =  "c:/users/frauke/downloads/hcp_task/"
if not os.path.isdir(HCP_DIR):
  os.mkdir(HCP_DIR)

# The data shared for NMA projects is a subset of the full HCP dataset
N_SUBJECTS = 339

# The data have already been aggregated into ROIs from the Glasesr parcellation
N_PARCELS = 360

# The acquisition parameters for all tasks were identical
TR = 0.72  # Time resolution, in sec

# The parcels are matched across hemispheres with the same order
HEMIS = ["Right", "Left"]

# Each experiment was repeated multiple times in each subject
N_RUNS_REST = 4
N_RUNS_TASK = 2

# Time series data are organized by experiment, with each experiment
# having an LR and RL (phase-encode direction) acquistion
BOLD_NAMES = [
  "rfMRI_REST1_LR", "rfMRI_REST1_RL",
  "rfMRI_REST2_LR", "rfMRI_REST2_RL",
  "tfMRI_MOTOR_RL", "tfMRI_MOTOR_LR",
  "tfMRI_WM_RL", "tfMRI_WM_LR",
  "tfMRI_EMOTION_RL", "tfMRI_EMOTION_LR",
  "tfMRI_GAMBLING_RL", "tfMRI_GAMBLING_LR",
  "tfMRI_LANGUAGE_RL", "tfMRI_LANGUAGE_LR",
  "tfMRI_RELATIONAL_RL", "tfMRI_RELATIONAL_LR",
  "tfMRI_SOCIAL_RL", "tfMRI_SOCIAL_LR"
]

# You may want to limit the subjects used during code development.
# This will use all subjects:
subjects = range(N_SUBJECTS)

## Create and update necessary files and subject-folders
Helper Functions

In [28]:
def mask_subs_no_error(error_files, subjects, HCP_DIR, bold_run):
    
    """Get list of subject with 0 errors in at least one condition 
       in task.

    Args:
      files_oI (list of strings) : list of condition .txt files to check ()
      subjects (list of int) : subjects t
      HCP_DIR(str): directory path
      bold_run(list): list of bold_runs ("tfMRI_WM_RL", "tfMRI_WM_LR")      
      
    Returns:
       2x excl_subs_**** (tuple) : list of unique subjects

    """

    warnings.filterwarnings("ignore", message="genfromtxt")


    files_oI = error_files        #currently ['2bk_err.txt', '2bk_nlr.txt']

    excl_subs_tfMRI_WM_RL = [] ##initialize list per task 
    excl_subs_tfMRI_WM_LR = []
    

    for t in bold_run: # iterate over task and subjects

        for sub in subjects:

            path = HCP_DIR + "/subjects/{}/EVs/{}".format(sub,t) #set path
            files = os.listdir(path)
            files = [file for file in files if file in files_oI] # choose files of interest from path

            for file in files:
                f = np.genfromtxt(path + "/" + file)

                if f.size == 0: # check for empty files in specific task
                    if t == "tfMRI_WM_RL":
                        excl_subs_tfMRI_WM_RL.append(sub) # append to respective list 
                    else:
                        excl_subs_tfMRI_WM_LR.append(sub)
                        
    #print(len(list(set(excl_subs_tfMRI_WM_RL)))) ###sanity check
    #print(len(list(set(excl_subs_tfMRI_WM_LR))))                    

    # return unique subjects per task, hint: if files_oI contains > 1 condition, doubles are possible  
    return (list(set(excl_subs_tfMRI_WM_RL)), list(set(excl_subs_tfMRI_WM_LR)))

In [27]:
mask_subs_no_error(['2bk_err.txt', '2bk_nlr.txt'], subjects, HCP_DIR, ['tfMRI_WM_RL','tfMRI_WM_LR'])

([0,
  11,
  16,
  19,
  27,
  34,
  39,
  41,
  42,
  46,
  47,
  52,
  55,
  58,
  59,
  67,
  70,
  72,
  75,
  81,
  87,
  94,
  98,
  100,
  103,
  105,
  108,
  112,
  117,
  118,
  121,
  122,
  124,
  136,
  139,
  142,
  144,
  146,
  152,
  160,
  167,
  197,
  198,
  200,
  208,
  211,
  215,
  218,
  220,
  221,
  230,
  231,
  238,
  240,
  252,
  255,
  257,
  262,
  264,
  269,
  275,
  278,
  279,
  281,
  283,
  289,
  293,
  294,
  307,
  309,
  310,
  314,
  318,
  319,
  322,
  323,
  329,
  334],
 [0,
  1,
  7,
  11,
  13,
  15,
  16,
  19,
  20,
  22,
  23,
  24,
  26,
  27,
  30,
  33,
  35,
  36,
  38,
  39,
  41,
  46,
  47,
  49,
  52,
  56,
  57,
  58,
  59,
  60,
  64,
  65,
  66,
  68,
  72,
  73,
  75,
  80,
  82,
  83,
  84,
  85,
  88,
  91,
  92,
  94,
  98,
  103,
  105,
  107,
  111,
  113,
  117,
  118,
  122,
  125,
  127,
  128,
  132,
  133,
  134,
  135,
  139,
  140,
  142,
  143,
  144,
  146,
  149,
  150,
  154,
  158,
  160,
  163,
  167,
  

In [14]:
def write_files(data, subject, bold_run, HCP_DIR, taskcondition):
    '''Write txt files from arrays, no matter whether they are empty
    Args:
        data: data to write in file
        subject(int): name of subject
        bold_run(str): name of the bold_run/task, e.g. wm_rl
        HCP_DIR(str): path which contains data
        taskcondition(str): new name of error/condition
    '''
    
    # Check data for size
    if data.size < 3: #if an error was made there would be at least one row for onset,duration,amplitude --> 3 int
        data = [] #set data equals empty
        #write an empty file in subjects folder of the the task with the errorcondition name
        np.savetxt(f"{HCP_DIR}/subjects/{subject}/EVs/{task_key}/{taskcondition}.txt",
               data, delimiter='\t')
    else: 
        if data.size == 3: #if data contains 3 columns
            data = data.reshape(1,3)
        #write file which contains data
        np.savetxt(f"{HCP_DIR}/subjects/{subject}/EVs/{task_key}/{taskcondition}.txt",
               data, fmt=['%.3f','%.1f', '%.d'], delimiter='\t')
        
def remove_omission_errors(HCP_DIR, subjects, bold_run, errconditions, nlrconditions):
    '''Remove omission errors from _bk_err.txt file.

    Args:
      HCP_DIR(str) : Path to target directory
      subjects(list): list of all subjects whos error files should be updated
      bold_run(list): list of task whos error files should be updated
      errconditions(list): list of error files which should be updated
      nlrconditions(list): list of nullresponse files which are used for comparison between false response and nullresponse
    Returns:
      New files '_bk_err_fresp.txt' in source directory 
  '''
    for subject in subjects: 
        for run in bold_run:
            #Load error files as numpy arrays 
            nlrconcat = np.zeros([1,3])
            ONSETS = []
            #loop through all errorconditions
            for i in np.arange(len(errconditions)):
                #print('subject {0}, task key {1}, i = {2}'.format(subject, task_key, i))
                errorcondition = errconditions[i] #set actual error condition
                err = np.genfromtxt(f"{HCP_DIR}/subjects/{subject}/EVs/{run}/{errorcondition}.txt") #load corresponding error file
                nlr = np.genfromtxt(f"{HCP_DIR}/subjects/{subject}/EVs/{run}/{nlrconditions[i]}.txt") #load corresponding null response file
                errorname = str(errconditions[i].split('_')[0]+'_fresp') #create the name of sorted error as false response
                #if there were zero null responses
                if nlr.size == 0:
                    #the complete error file contains just false reponses, so the data to write in new file equals the errorfile
                    data = err
                    #write new error- false reponse file
                    write_files(data, subject, task_key, HCP_DIR, errorname)
                    #go to next condition
                    continue
                # if there was one null reponse AND just one error in the other error file
                elif nlr.ndim == 1 and err.ndim == 1:
                    #those error file just contains a null response which should be deleted
                    onsets = nlr[0]
                    mask = np.isin(err, nlr)[0]
                    data = np.delete(err, np.where(mask==True), axis=0)
                    #data = []
                    write_files(data, subject, task_key, HCP_DIR, errorname)
                # if there was just one null response but more total errors
                elif nlr.ndim == 1 and err.ndim > 1:
                    onsets = nlr[0]
                    mask = np.isin(err, nlr)[:,0] # create mask for null reponse in other errors
                    data = np.delete(err, np.where(mask==True), axis=0) #delete masked data and create new (false response) data
                    #write false response data in new file
                    write_files(data, subject, task_key, HCP_DIR, errorname)
                #else if there were more than one null response and more than one total errors
                else:
                    onsets = nlr[:,0]
                    mask = np.isin(err, nlr)[:,0] #create mask for null response in total errors
                    data = np.delete(err, np.where(mask==True), axis=0) #delete masked data and create new (false response) data
                    #write false response data in new file
                    write_files(data, subject, task_key, HCP_DIR, errorname)
                ONSETS = np.append(ONSETS, onsets)

            all_err = np.genfromtxt(f"{HCP_DIR}/subjects/{subject}/EVs/{run}/all_bk_err.txt")
            errorcondition = 'all_bk_err'
            errorname = str(errorcondition.split('_')[0]+'_fresp')
            #if there was no error at all
            if all_err.size == 0:
                data = all_err #false response data is empty as well
                write_files(data, subject, task_key, HCP_DIR, errorname)
                continue
            #if amount of total errors equals 1
            elif all_err.ndim == 1:
                mask = np.isin(all_err[0], ONSETS) #mask total errors with the onsets of the null responses
                data = np.delete(all_err, np.where(mask==True), axis=0) #delete null responses and create new (false response) data
                #write false responses in new file
                write_files(data, subject, task_key, HCP_DIR, errorname)
            #if amount of total errors are more than 1
            else:
                mask = np.isin(all_err[:,0], ONSETS) #mask total errors with the onsets of the null responses
                data = np.delete(all_err, np.where(mask==True), axis=0) #delete null responses and create new (false response) data
                #write false responses in new file
                write_files(data, subject, task_key, HCP_DIR, errorname)

In [23]:
def return_X_sorted(HCP_DIR, subject, bold_run):
    '''Loads _cor.txt and _err.txt files, concatenates and sorts after time
    
    Args:
    - HCP_DIR (str)  : path of directory 
    - subject (int)  : current subject 
    - bold_run (str)  : task/bold run
    
    Returns: 
    - X (np.ndarray) : 80x2 ndarray with columns time stamp and errors (1 = error)
    '''
    #load all responses in two variables
    corr = np.genfromtxt(f"{HCP_DIR}/subjects/{subject}/EVs/{bold_run}/all_bk_cor.txt")
    err =  np.genfromtxt(f"{HCP_DIR}/subjects/{subject}/EVs/{bold_run}/all_bk_err.txt")
    #stack all onset times nebeneinander
    corr = np.hstack((corr[:,0].reshape(-1,1), np.zeros_like(corr[:,0].reshape(-1,1))))
    err = np.hstack((err[:,0].reshape(-1,1), np.ones_like(err[:,0]).reshape(-1,1)))
    #stack both responses (correct and error) into one array
    X = np.vstack((corr, err))
    #sort onset times after size
    X = X[X[:,0].argsort()]
    
    return X 

def diff_stimuli_types(HCP_DIR, subjects, stimuli, bold_run, task_keys):
    '''
        Takes all stimuli and sort them after stimuli type. Writes onset times of different stimuli type in a several files.
        
        Args:
            HCP_DIR (str): directory of data
            subjects(array): array of all subjects
            stimuli(list): list of stimuli for which onset files should be written
            bold_run(list): list of all run/task for which onset files for different stimuli should be written
            task_keys(list): list of tasks for which onset files for different stimuli should be written
        saves new stimuli onset files
    '''
    cue_duration = 2.565
    len_block = 10
    temp_zeros = np.zeros([80,4])
    for subject in subjects[:2]:
        for run in bold_run[:1]:
            X = return_X_sorted(HCP_DIR=HCP_DIR, subject=subject, bold_run=run)
            
            for task_key in task_keys:
                temp_zeros = np.zeros([80,4])
                for index, stimulus in enumerate(stimuli):
                    stim_name = task_key + stimulus
                    stim = np.genfromtxt(f"{HCP_DIR}/subjects/{subject}/EVs/{run}/{stim_name}.txt")
                    #Search for index of start value (min distance between stim onset + 
                    # cue duration and start of block in onset times of X)
                    min_index = np.argmin(np.abs(X[:,0] - (stim[0] + cue_duration)))
                    # Create new column of zeros and ones 
                    temp_zeros[min_index:min_index+len_block, index] = 1
                    T = np.hstack((X[:,0].reshape(-1,1), temp_zeros))

                    #Create new file with time stamps 
                    M = np.hstack((X[min_index : min_index+len_block, 0].reshape(-1,1), np.full((len_block,1),2.5),
                                   np.full((len_block, 1), 1)))
                    # Optional if we want to include the first line 
                    M = np.vstack((stim, M))
                    # Write file in source directory
                    np.savetxt(f"{HCP_DIR}/subjects/{subject}/EVs/{run}/{stim_name}_ts.txt",
                               M, fmt=['%.3f','%.1f', '%.d'], delimiter='\t')
                
                #Write T into file
                np.savetxt(f"{HCP_DIR}/subjects/{subject}/EVs/{run}/all_{task_key}ts.txt",
                            T, fmt=['%.3f','%.d', '%.d','%.d', '%.d'], delimiter='\t')

In [24]:
diff_stimuli_types(HCP_DIR, subjects,['faces', 'body', 'tools', 'places'],["tfMRI_WM_RL", "tfMRI_WM_LR"],['0bk_', '2bk_'])

## Data Loading
Helper Functions

In [45]:
def get_image_ids(name):
  """Get the 1-based image indices for runs in a given experiment.

    Args:
      name (str) : Name of experiment ("rest" or name of task) to load
    Returns:
      run_ids (list of int) : Numeric ID for experiment image files

  """
  run_ids = [
    i for i, code in enumerate(BOLD_NAMES, 1) if name.upper() in code]
  if not run_ids:
    raise ValueError(f"Found no data for '{name}''")
  return run_ids

def load_evs(subject, name, condition):
  """Load EV (explanatory variable) data for one task condition.

  Args:
    subject (int): 0-based subject ID to load
    name (str) : Name of task
    condition (str) : Name of condition

  Returns
    evs (list of dicts): A dictionary with the onset, duration, and amplitude
      of the condition for each run.

  """
  evs = []
  for id in get_image_ids(name):
    task_key = BOLD_NAMES[id-1]
    ev_file = f"{HCP_DIR}subjects/{subject}/EVs/{task_key}/{condition}.txt"
    ev = dict(zip(["onset", "duration", "amplitude"], np.genfromtxt(ev_file).T))
    evs.append(ev)
  return evs


In [30]:
def load_timeseries(subject, name, runs=None, concat=True, remove_mean=True):
  """Load timeseries data for a single subject.
  
  Args:
    subject (int): 0-based subject ID to load
    name (str) : Name of experiment ("rest" or name of task) to load
    run (None or int or list of ints): 0-based run(s) of the task to load,
      or None to load all runs.
    concat (bool) : If True, concatenate multiple runs in time
    remove_mean (bool) : If True, subtract the parcel-wise mean

  Returns
    ts (n_parcel x n_tp array): Array of BOLD data values

  """
  # Get the list relative 0-based index of runs to use
  if runs is None:
    runs = range(N_RUNS_REST) if name == "rest" else range(N_RUNS_TASK)
  elif isinstance(runs, int):
    runs = [runs]

  # Get the first (1-based) run id for this experiment 
  offset = get_image_ids(name)[0]

  # Load each run's data
  bold_data = [
      load_single_timeseries(subject, offset + run, remove_mean) for run in runs
  ]

  # Optionally concatenate in time
  if concat:
    bold_data = np.concatenate(bold_data, axis=-1)

  return bold_data


def load_single_timeseries(subject, bold_run, remove_mean=True):
  """Load timeseries data for a single subject and single run.
  
  Args:
    subject (int): 0-based subject ID to load
    bold_run (int): 1-based run index, across all tasks
    remove_mean (bool): If True, subtract the parcel-wise mean

  Returns
    ts (n_parcel x n_timepoint array): Array of BOLD data values

  """
  bold_path = f"{HCP_DIR}subjects/{subject}/timeseries"
  bold_file = f"bold{bold_run}_Atlas_MSMAll_Glasser360Cortical.npy"
  ts = np.load(f"{bold_path}/{bold_file}")
  if remove_mean:
    ts -= ts.mean(axis=1, keepdims=True)
  return ts

In [31]:
def condition_frames(run_evs, skip=0):
    """Identify timepoints corresponding to a given condition in each run.
    Args:
        run_evs (list of dicts) : Onset and duration of the event, per run
        skip (int) : Ignore this many frames at the start of each trial, to account
        for hemodynamic lag
    
    Returns:
        frames_list (list of 1D arrays): Flat arrays of frame indices, per run
        
    """
    frames_list = []
    for ev in run_evs:
        try:
            # Determine when trial starts, rounded down
            start = np.floor(ev["onset"] / (TR/10)).astype(int)
            
            #Use trial duration to determine how many frames to include for trial
            duration = np.ceil(ev["duration"] / (TR/10)).astype(int)
            
            # Take the range of frames that correspond to this specific trial
            if type(start) == np.ndarray:
                frames = [s + np.arange(skip, d) for s, d in zip(start, duration)]
            else:
                frames = [start+np.arange(skip,duration)]
            
            frames_list.append(np.concatenate(frames))
            #print('ev-file is not empty')
        except KeyError:
            print('ev-file is empty')

    return frames_list

def condition_frames_iti(run_evs, skip=0):
    """Identify starting timepoint and duration of an inter-trial-interval corresponding to a given condition in each run.
    Args:
        run_evs (list of dicts) : Onset and duration of the event, per run
        skip (int) : Ignore this many frames at the start of each trial, to account
        for hemodynamic lag
    
    Returns:
        frames_list (list of 1D arrays): Flat arrays of frame indices, per run
        
    """
    frames_list = []
    for ev in run_evs:
        try:
            # Determine when trial starts, rounded down
            start = np.floor((ev["onset"]+2) / (TR/10)).astype(int) 
            
            #Use trial duration to determine how many frames to include for trial        
            duration = np.ceil((np.ones(len(ev['duration']))*0.5) / (TR/10)).astype(int)
            
            # Take the range of frames that correspond to this specific trial
            if type(start) == np.ndarray:
                frames = [s + np.arange(skip, d) for s, d in zip(start, duration)]
            else:
                frames = [start+np.arange(skip,duration)]
            
            frames_list.append(np.concatenate(frames))
            #print('ev-file is not empty')
        except KeyError:
            print('ev-file is empty')

    return frames_list

## Preprocessing and GLM-Fitting
Helper Functions

In [32]:
def make_design_matrix(subject,task,conditions,sample_rate):
    """
    Creates a design matrix for given tasks and conditions.
    
    Args:
        task: string of specific task of experiment
        conditions: list of conditions, which should be included in design matrix
        sample_rate: int of how much the frames should be upsampled
    """
    
    #ignore warnings from genfromtxt
    warnings.filterwarnings("ignore", message="genfromtxt")
    #create instances of events and design_matrix
    ts = load_single_timeseries(subject,get_image_ids(task)[0])
    design_matrix = np.zeros((ts.shape[1]*sample_rate,len(conditions)))
    #loop through all conditions and set a 1 in design matrix for every time point condition is fullfilled
    for n, cond in enumerate(conditions):
        frames= []
        ev = [load_evs(subject,task,cond)]
        frames = condition_frames_iti(ev[0],skip=0)
        #frames = condition_frames(ev[0],skip=0)
        design_matrix[frames,n] = 1
    #loop through design matrix columns and delete design matrix if at least one row has no entries
    for i in range(len(conditions)):
        if np.sum(design_matrix,axis=0)[i]==0:
            design_matrix = []
            print(subject,' has not made enough errors')
            break
    return design_matrix

def create_hrf(tr,sampling_rate):
    '''
        Creates an HRF with specific upsampling rate
        
        Arg:
            tr = repition time of MRT-Scanner
            sampling_rate (int) = factor with which hrf should be oversampled on basis of repition time
    '''
    #creates an hrf
    canon_hrf = glover_hrf(tr=tr, oversampling=sampling_rate)
    
    # Define timepoints corresponding to HRF
    t_hrf = np.linspace(0, np.floor(32/tr).astype(int), np.round(32/tr * sampling_rate).astype(int), endpoint=False)
    
    return canon_hrf, t_hrf

In [33]:
def hrf_convo(design_matrix,bf):
    """
    Takes designmatrix upsampled to TR/TReso and convolves it with canonical HRF
    
    Args
    design_matrix: C * VolTreso ndarray; indicates closest Tr/TReso timepoint where an event started for specific regressor
    bf: basis function - will usually be the hrf; 1D array

    Returns
    design_matrix_conv: CVol array with predicted BOLD response
    """
    
    try:
        design_matrix_pad=np.pad(design_matrix,((0,len(bf)),(0, 0))) # pad with zeros, so we can fit the hrf also for very late events
        design_matrix_conv=np.empty(design_matrix_pad.shape) # prepare convolved design matrix
    
        for C in range(len(design_matrix[1])): # for each condition C
            design_matrix_conv[:,C]=np.convolve(design_matrix_pad[:len(design_matrix_pad[0])-(len(bf)+1),C],bf) #convolute with design with bf

        design_matrix_downsampled=design_matrix_conv[:len(design_matrix_pad[0])-(len(bf)+2),:] # cut back the padded stuff

        design_matrix_downsampled=design_matrix_downsampled[0::10,:] # pick every time-point where we have a volume
    
        return design_matrix_downsampled
    except ValueError:
        print('subject has no design matrix')
    

In [34]:
def fitGLM(design_matrix_GLM,TimeSeries):
    """
    Takes design matrix and timeseries data and applies MLE to generate beta-estimates
    Args: 
        design_matrix_GLM: C*Vol array with predicted BOLD signal per volume per regressor (MAY NOT CONTAIN 0-SUM VECTORS)
        TimeSeries: Region*Vol array with actual BOLD signal averaged across regions
    Return:
        beta: 
    """
    X=design_matrix_GLM
    ts=TimeSeries # get time-series array
    beta=np.empty((np.size(TimeSeries,axis=0),np.size(X,axis=1)))
    for R in range(np.size(ts,axis=0)):
        beta[R,:] = np.linalg.inv(X.T @ X) @ X.T @ ts[R,:].T
    return beta 

In [35]:
# set some start variables
# the shape of the design matrix is based on the script "DataAnalysis_Errorss"
mat_shape = (4050, 2)
design_matrix = np.array([[1, 0, 1, 0, 0, 0],
                [0, 1, 0, 1, 0 ,0],
                [0, 0, 0, 0, 1, 1],
                [1, 1, 1, 1, 0, 0]])

def test_orthogonality(design_matrix, warn = False):
    
    """
    takes a design martix and computed the correlation between conditions
    
    input:
    design_matrix = m x n np.array with condition columns and time line as a row
    
    output:
    orth_mat = np.array of size n x n indicating the orthogonality of each condition to each other condition
    """
    
    # initialize the output matrix
    design_dim = design_matrix.shape
    orth_mat = np.zeros((design_dim[1], design_dim[1]))
    
    # two vectors are orthogonal, if their dot product is zero
    # loop through all conditions
    for condition in range(0,design_dim[1]):
        cond_1 = design_matrix[:,condition]
        # compare to all conditions
        for compare in range(0, design_dim[1]):
            cond_2 = design_matrix[:,compare]
            
            orth_mat[condition,compare] = cond_1@cond_2/design_dim[1]
                        
    # throw a warning for conditions that are not orthogonal
    if warn:
        warnings = np.where(orth_mat>0)
        for pos in range(0,len(warnings[0])):
            check_pos = np.zeros(len(warnings))

            for dim in range(0,len(warnings)):
                check_pos[dim] = warnings[dim][pos]

            print(check_pos)
            if not (check_pos[0]== check_pos[1]):
                print('Watch out, the matrix is not orthogonal for conditons {}'.format(check_pos))   
    
    return orth_mat

## First level analysis 
Helper Functions

In [36]:
def selective_average(timeseries_data, ev, skip=0):
  """Take the temporal mean across frames for a given condition.

  Args:
    timeseries_data (array or list of arrays): n_parcel x n_tp arrays
    ev (dict or list of dicts): Condition timing information
    skip (int) : Ignore this many frames at the start of each trial, to account
      for hemodynamic lag

  Returns:
    avg_data (1D array): Data averagted across selected image frames based
    on condition timing

  """
  # Ensure that we have lists of the same length
  if not isinstance(timeseries_data, list):
    timeseries_data = [timeseries_data]
  if not isinstance(ev, list):
    ev = [ev]
  if len(timeseries_data) != len(ev):
    raise ValueError("Length of `timeseries_data` and `ev` must match.")

  # Identify the indices of relevant frames
  frames = condition_frames(ev)

  # Select the frames from each image
  selected_data = []
  for run_data, run_frames in zip(timeseries_data, frames):
    selected_data.append(run_data[:, run_frames])

  # Take the average in each parcel
  avg_data = np.concatenate(selected_data, axis=-1).mean(axis=-1)

  return avg_data

## Workflows

In [ ]:
#initialize basic parameters
bold_run = ['tfMRI_WM_RL','tfMRI_WM_LR']
bold_ids = [get_image_ids('wm_rl')[0],get_image_ids('wm_lr')[0]]
task_keys = ['0bk_','2bk_']
stimuli = ['faces', 'body', 'tools', 'places']
#create error files with false response and null responses respectively
remove_omission_errors(HCP_DIR, subjects,bold_run, ['0bk_err','2bk_err'],['0bk_nlr','2bk_nlr'])
#initialize new error files
error_files = ['2bk_err_fresp','2bk_nlr']
#create lists of subjects with at least one error of each error type in both bold runs
subjects_rl, subjects_lr = mask_subs_no_error(error_files, subjects, HCP_DIR, bold_run)
#create onset files for each specific stimuli type
diff_stimuli_types(HCP_DIR, subjects_rl, stimuli, bold_run[0], task_keys)
diff_stimuli_types(HCP_DIR, subjects_lr, stimuli, bold_run[1], task_keys)
#make design matrix


In [ ]:
task = 'wm_rl'
bold_run = get_image_ids(task)[0]
sub = 338
design_matrix = make_design_matrix(sub,task,['2bk_err2','2bk_nlr'],10)
design_matrix_GLM = hrf_convo(design_matrix, canon_hrf)
TimeSeries = load_single_timeseries(sub,bold_run)
GLM = fitGLM(design_matrix_GLM,TimeSeries)

In [50]:
bold_ids = [get_image_ids('wm_rl')[0],get_image_ids('wm_lr')[0]]
bold_ids

[7, 8]

In [48]:
get_image_ids['wm_rl'][0]

TypeError: 'function' object is not subscriptable